In [1]:
import pandas as pd
import tqdm

from Wayfair_Scrapy import *
link = 'https://www.wayfair.ca/graphql'

In [64]:
from Wayfair_Scrapy import *
link = 'https://www.wayfair.com/graphql'
hash = 'e67bfa2d6bd7bcf6d8c351a68c4e18aa#77c656652c1c4814baceb2546e8c8ff9#cb05724590826da5c40607d51d79e079#28c908fd3aa88467ecab787c0251bde9'
sku = 'W009948296'
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'use-web-hash': 'true'
}

sp = requests.post(
    link,
    json={"variables":{"sku":"W005967825","additionalSkus":["W005967825","W005001636","W007660354","W006789853","W006689508"],"selectedOptionsIds":["1894013204"],"fullyConfigureOptions":False}},
    params={"hash":hash},
    headers=headers)
print(sp.url)
content = sp.json()

https://www.wayfair.com/v/captcha/show?goto=https%3A%2F%2Fwww.wayfair.com%2Fgraphql%3Fhash%3De67bfa2d6bd7bcf6d8c351a68c4e18aa%252377c656652c1c4814baceb2546e8c8ff9%2523cb05724590826da5c40607d51d79e079%252328c908fd3aa88467ecab787c0251bde9&px=1&captcha_status=0


JSONDecodeError: Expecting value: line 1 column 2 (char 1)

In [13]:

def get_all_combine(categories):
    combine_list = []
    cate_dict = {}
    for category in categories:
        options = category['productOptions']
        categoryName = category['categoryName']
        list_by_cate = []
        for option in options:
            list_by_cate.append(option['optionId'])
            value = "{}:{}".format(categoryName, option['name'])
            cate_dict[str(option['optionId'])] = value
        combine_list.append(list_by_cate)

    all_list = lists_combination(combine_list)
    new_all_list = []
    if all_list:
        for combination in all_list:
            new_all_list.append(str(combination).split('|'))

    return new_all_list, cate_dict


In [3]:
def sort_list(list):
    exceptions = []
    for i in list:
        i.sort()
        new_i = []
        for g in i:
            new_i.append(str(g))
        exceptions.append(new_i)
    return exceptions

In [4]:
def getinfo(sku,optionId_list):
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.54',
        'use-web-hash': 'true'
    }
    link = 'https://www.wayfair.ca/graphql'
    sp = requests.post(
        link,
        json={"variables":{"sku":sku,"additionalSkus":[sku],"selectedOptionsIds":optionId_list if optionId_list else []}},
        params={"hash":"e67bfa2d6bd7bcf6d8c351a68c4e18aa#77c656652c1c4814baceb2546e8c8ff9#cb05724590826da5c40607d51d79e079#28c908fd3aa88467ecab787c0251bde9"},
        headers=headers)
    content = sp.json()
    if content['data']['product']['optionMatchedProducts']:
        optionMatchedProduct = content['data']['product']['optionMatchedProducts'][0]
    else:
        return {'listPrice':'-','customerPrice':'-','stock':'-','sale_tag':'-'}
    try:
        if optionMatchedProduct['pricing']['listPrice'] is None:
            listPrice = '-'
        else:
            listPrice = optionMatchedProduct['pricing']['listPrice']['unitPrice']['value']
        customerPrice = optionMatchedProduct['pricing']['customerPrice']['unitPrice']['value']
        stock = optionMatchedProduct['inventory']['stockStatus']
        sale_tag = optionMatchedProduct['pricing']['priceBlockElements'][0]['appliedPromotion']
        return {'listPrice':listPrice,'customerPrice':customerPrice,'stock':stock,'sale_tag':sale_tag}
    except:
        return {'listPrice':'-','customerPrice':'-','stock':'-','sale_tag':'-'}

In [ ]:
def getqpl(info_list,sku):
    customerReviews = []
    optionExceptions = []
    optionCategories = []

    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.54',
        'use-web-hash': 'true'
    }
    sp = requests.post(
        link,
        json={"variables":{"sku":sku}},
        params={"hash":"9019b6158a2d29556a702eee5194ad24#7f637117145182d911071391763e37cc#77c656652c1c4814baceb2546e8c8ff9#cb05724590826da5c40607d51d79e079#08c5d1b7acbca00d37d05400adc9bee8#28c908fd3aa88467ecab787c0251bde9"},
        headers=headers)
    content = sp.json()
    # with open(r'C:\Users\Admin\data.json', 'w', encoding='utf-8') as f:
    #     json.dump(content, f, ensure_ascii=False, indent=4)
    if 'errors' in content:
        info_dict = {"SKU":sku,"title":'-',"full_name":'-',"optionid":'-',"stock":'-',"customerPrice":'-',"listPrice":'-',"sale_tag":'-',"rate":'-',"reviews":'-',"url":'-'}
        for key in info_dict:
            info_list[key].append(info_dict[key])
        return
    items = content['data']['product']['crossSell']['items']
    if items:
        n=0
        item = items[n]
        while items[n]['sku'] != sku:
            n+=1
            if n == len(items):
                info_dict = {"SKU":sku,"title":'-',"full_name":'-',"optionid":'-',"stock":'-',"customerPrice":'-',"listPrice":'-',"sale_tag":'-',"rate":'-',"reviews":'-',"url":'-'}
                for key in info_dict:
                    info_list[key].append(info_dict[key])
                return
            item = items[n]
        customerReviews = item['customerReviews']
        optionExceptions = item['optionExceptions']
        optionCategories = item['options']['optionCategories']
        title = item['name']
        url = item['url']


    else:
        info_dict = {"SKU":sku,"title":'-',"full_name":'-',"optionid":'-',"stock":'-',"customerPrice":'-',"listPrice":'-',"sale_tag":'-',"rate":'-',"reviews":'-',"url":'-'}
        for key in info_dict:
            info_list[key].append(info_dict[key])
        return

    rate = customerReviews['averageRatingValue']
    reviews = customerReviews['ratingCount']

    new_all_list, cate_dict = get_all_combine(optionCategories)
    print(new_all_list)
    new_all_list = sort_list(new_all_list)
    exceptions = sort_list(optionExceptions)

    if new_all_list:
        for combin in new_all_list:
            if combin not in exceptions:
                info_dict = getinfo(sku,combin)
                full_name_list = []
                for c in combin:
                    full_name_list.append(cate_dict[c])
                info_dict['full_name'] = '&'.join(full_name_list)
                info_dict['optionid'] = '&'.join(combin)

                info_dict['SKU'] = sku
                info_dict['rate'] = rate
                info_dict['reviews'] = reviews
                info_dict['title'] = title
                info_dict['url'] = url+'?&piid='+ info_dict['optionid']
                for key in info_dict:
                    info_list[key].append(info_dict[key])
    else:
        info_dict = getinfo(sku,[])
        info_dict['full_name'] = title
        info_dict['optionid'] = '-'
        info_dict['SKU'] = sku
        info_dict['rate'] = rate
        info_dict['reviews'] = reviews
        info_dict['title'] = title
        info_dict['url'] = url+'?&piid='+ info_dict['optionid']
        for key in info_dict:
            info_list[key].append(info_dict[key])

In [2]:
from Wayfair_Scrapy import *
csv_path_US = r'C:\Users\Admin\Nutstore\1\「晓望集群」\S数据分析\Wayfair爬虫\SKU_list_CA.csv'
data = read_src(csv_path_US)
df_dict = {'sku':[],'max':[],'min':[],'avg':[]}
hash_ca = '9019b6158a2d29556a702eee5194ad24#7f637117145182d911071391763e37cc#77c656652c1c4814baceb2546e8c8ff9#cb05724590826da5c40607d51d79e079#08c5d1b7acbca00d37d05400adc9bee8#28c908fd3aa88467ecab787c0251bde9'
hash_us = 'fee62fb54d3d50424e02336d2dd8521b#7f637117145182d911071391763e37cc#77c656652c1c4814baceb2546e8c8ff9#cb05724590826da5c40607d51d79e079#08c5d1b7acbca00d37d05400adc9bee8#28c908fd3aa88467ecab787c0251bde9'

for sku in data:
    print(sku[0])
    link = 'https://www.wayfair.ca/graphql'
    hash_us = hash_ca
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.54',
        'use-web-hash': 'true'
    }
    try:
        sp = requests.post(
            link,
            json={"variables":{"sku":sku[0]}},
            params={"hash":hash_us},
            headers=headers)
        content = sp.json()
        items = content['data']['product']['crossSell']['items']
        dic = {}
        data_dict = {}

        if items:
            for n in range(len(items)):
                item = items[n]
                ratingCount = item['customerReviews']['ratingCount']
                price = item['pricing']['customerPrice']['unitPrice']['value']
                data_dict[price] = ratingCount
        dic.update(dict(sorted(data_dict.items(), key=lambda x: x[1], reverse=True)[:5]))
        value = 0
        max = 0
        min = 99999
        count_total = 0
        for key in dic:
            price = float(key)
            review_count = float(dic[key])
            count_total=review_count+count_total
            value = price*review_count+value
            if price>max:
                max = price
            if price<min:
                min = price

        avg = value/count_total
    except:
        avg = max = min = '-'
    print(avg)
    item_dict = {'sku':sku[0],'max':max,'min':min,'avg':avg}
    for key in df_dict:
        df_dict[key].append(item_dict[key])
pd.DataFrame(df_dict).to_csv(r'C:\Users\Admin\Nutstore\1\「晓望集群」\S数据分析\Wayfair爬虫\output_CA.csv',index_label=None)

C009292044
-
C009292718
-
C009295905
-
C003257754
-
C004163874
265.11091791703444
C000446162
218.2841820442191
TNFI2492
-
TNFI2481
180.76269526248402
TNFI2483
180.76269526248402
TNFI2488
218.2841820442191
TNFI2493
218.2841820442191
TNFI2466
218.2841820442191
TNFI2478
191.66154574132491
TNFI2463
191.66154574132491
TNFI2474
191.66154574132491
TNFI2468
218.2841820442191
TNFI2464
218.2841820442191
TNFI2486
218.2841820442191
TNFI2476
218.2841820442191
TNFI2470
-
TNFI2469
-
TNFI2479
-
TNFI2490
-
TNFI2477
-
TNFI2484
-
TNFI2465
-
TNFI2471
218.2841820442191
TNFI2487
218.2841820442191
TNFI2489
232.23658755274266
TNFI2491
218.2841820442191
TNFI2480
218.2841820442191
TNFI2482
218.2841820442191
TNFI2473
218.2841820442191
TNFI2494
218.2841820442191
C003315737
915.3154
C004166423
190.98375664187037
C003355389
241.5582989690722
C003356075
398.97118962887646
C003356430
218.84573476702508
C003356114
212.52774992539543
C003355304
268.2271794871795
C003160444
328.32119577517915
C001327451
126.99
C00056456

In [1]:
from Wayfair_Scrapy_rd import stock_contrast
stock_contrast('US')

执行内容:{'msgtype': 'text', 'text': {'content': '今日断货SKU：\nBOGDAN MUSTARD\n  GORDAN RED\n  GORDAN WHITE\n  Hailey 24 Inch Brown 2Pcs\n  POUCHU LMKZ\n  RONDAE BLOSSOM\n  Scargill Beige\n  TRAPP PINK\n  TUDOR GREEN\n  VUSTU 24 RED METAL BAR STOOL Small Pallet\n  VUSTU 24 SILVER METAL BAR STOOL Small Pallet\n  WILONA DARK GREY\n  ZOLNES 24 GUN 4PCS\n  ZOLNES 24 GUN 2PCS\n  ZOMBA FABRIC BLACK\n  nan\n\n今日恢复供货SKU：\nGESSEY\n  Option������\n  KAROMI PU BLACK\n  TALYA', 'mentioned_mobile_list': ['18062351119']}}, 参数:{"errcode":0,"errmsg":"ok"}
webhook 发送结果:{"errcode":0,"errmsg":"ok"}
{"errcode":0,"errmsg":"ok"}
webhook 发出完毕: {"errcode":0,"errmsg":"ok"}


In [70]:
pd.DataFrame(df_dict).to_csv(r'C:\Users\Admin\Nutstore\1\「晓望集群」\S数据分析\Wayfair爬虫\output_CA_20230213.csv',index_label=None)

In [12]:
from Wayfair_Scrapy_rd import mapping_sku
csv_path1 = r'C:\Users\Admin\Nutstore\1\「晓望集群」\S数据分析\Wayfair爬虫\Wayfair_PriceOutput_20230115_CA.csv'
mapping_sku(
    csv_path1,
    r'C:\Users\Admin\Nutstore\1\「晓望集群」\S数据分析\Wayfair爬虫\SKU_Mapping_{}.csv'.format('CA'),
    'optionid', 'partner_number')

,SKU,title,full_name,optionid,stock,customerPrice,listPrice,sale_tag,rate,reviews,url,partner_number
0,C009295905,-,-,C009295905,-,-,-,-,-,-,-,AAREN PATTERN B
1,C009292044,-,-,C009292044,-,-,-,-,-,-,-,AAREN PATTERN C
2,C003257754,-,-,C003257754,-,-,-,-,-,-,-,ABELARDO THREE DOORS
3,C009292718,-,-,C009292718,-,-,-,-,-,-,-,AAREN PATTERN A
4,TNFI2492,-,-,TNFI2492,-,-,-,-,-,-,-,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1635,C004976971,Palomo Upholstered Dining Chair,Body Fabric:Rose Velvet &Leg Colour:Brown,1264750009&1264750027,IN_STOCK,189.99,296.99,SALE,4.633676093,3112,https://www.wayfair.ca/furniture/pdp/wade-loga...,NaN
1636,C004976971,Palomo Upholstered Dining Chair,Body Fabric:Yellow Fabric &Leg Colour:Black,1264750000&1264750028,IN_STOCK,187.99,339.99,SALE,4.633676093,3112,https://www.wayfair.ca/furniture/pdp/wade-loga...,NaN
1637,C004976971,Palomo Upholstered Dining Chair,Body Fabric:Aqua Green Velvet&Leg Colour:Brown,1264750011&1264750027,OUT_OF_STOCK,166.99,296.99,SALE,4.633676093,3112,https://www.wayfair.ca/furniture/pdp/wade-loga...,NaN
1638,C004976971,Palomo Upholstered Dining Chair,Body Fabric:Black&Leg Colour:Black,1264750025&1264750028,OUT_OF_STOCK,265.99,373.98,SALE,4.633676093,3112,https://www.wayfair.ca/furniture/pdp/wade-loga...,NaN


In [ ]:

info_list = {"SKU":[],"title":[],"full_name":[],"optionid":[],"stock":[],"customerPrice":[],"listPrice":[],"sale_tag":[],"rate":[],"reviews":[],"url":[]}
getqpl(info_list,'C003153110')
print(info_list)

In [7]:
csv_path_US = r'C:\Users\Admin\Nutstore\1\「晓望集群」\S数据分析\Wayfair爬虫\SKU_list_CA.csv'
data = read_src(csv_path_US)
info_list = {"SKU":[],"title":[],"full_name":[],"optionid":[],"stock":[],"customerPrice":[],"listPrice":[],"sale_tag":[],"rate":[],"reviews":[],"url":[]}

while data is not None:
    miss_list = []
    for sku in data:
        print('{},{}/{}'.format(sku[0],data.index(sku),len(data)))
        try:
            getqpl(info_list,sku[0])
        except:
            traceback.print_exc()
            miss_list.append(sku)
    if miss_list:
        data = miss_list
        sleep(600)
    else:
        data = None
    print(data)


C009292044,0/871
C009292718,1/871
C009295905,2/871
C003257754,3/871
C004163874,4/871
[['206486407'], ['206486404']]
C000446162,5/871
[['2056264327']]
TNFI2492,6/871
TNFI2481,7/871
[]
TNFI2483,8/871
[]
TNFI2488,9/871
[]
TNFI2493,10/871
[]
TNFI2466,11/871
[['79136777'], ['79136776']]
TNFI2478,12/871
[]
TNFI2463,13/871
[]
TNFI2474,14/871
[['79136738'], ['79136737']]
TNFI2468,15/871
[]
TNFI2464,16/871
TNFI2486,17/871
[]
TNFI2476,18/871
[['79622551'], ['79622552']]
TNFI2470,19/871
TNFI2469,20/871
TNFI2479,21/871
TNFI2490,22/871
TNFI2477,23/871
TNFI2484,24/871
TNFI2465,25/871
TNFI2471,26/871
[]
TNFI2487,27/871
[]
TNFI2489,28/871
[]
TNFI2491,29/871
[]
TNFI2480,30/871
[]
TNFI2482,31/871
[]
TNFI2473,32/871
[['79090579'], ['79090604']]
TNFI2494,33/871
[]
C003315737,34/871
[['526751987'], ['526751948'], ['526751949']]
C004166423,35/871
[['756610302'], ['756610303']]
C003355389,36/871
[['948192255'], ['948192252']]
C003356075,37/871
[['2071381559'], ['2071381556']]
C003356430,38/871
[['1107277292'

Traceback (most recent call last):
  File "<ipython-input-7-4c37f69e3b33>", line 10, in <module>
    getqpl(info_list,sku[0])
  File "<ipython-input-6-6d2361debb84>", line 87, in getqpl
    info_dict = getinfo(sku,combin)
  File "<ipython-input-6-6d2361debb84>", line 12, in getinfo
    content = sp.json()
  File "C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\requests\models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\simplejson\__init__.py", line 525, in loads
    return _default_decoder.decode(s)
  File "C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\simplejson\decoder.py", line 370, in decode
    obj, end = self.raw_decode(s)
  File "C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\simplejson\decoder.py", line 400, in raw_decode
    return self.scan_once(s, idx=_w(s, idx).end())
simplejson.errors.JSONDecodeError: Expecting value: line 1 column 1 (

C003152949,680/871
[['777194248'], ['777194249'], ['798441352'], ['798441353'], ['798441354']]
C003153110,681/871
[['1476443754'], ['1600033823'], ['1476443757'], ['1600033822'], ['1587307974'], ['1587307977'], ['1600033820']]


Traceback (most recent call last):
  File "<ipython-input-7-4c37f69e3b33>", line 10, in <module>
    getqpl(info_list,sku[0])
  File "<ipython-input-6-6d2361debb84>", line 87, in getqpl
    info_dict = getinfo(sku,combin)
  File "<ipython-input-6-6d2361debb84>", line 12, in getinfo
    content = sp.json()
  File "C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\requests\models.py", line 897, in json
    return complexjson.loads(self.text, **kwargs)
  File "C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\simplejson\__init__.py", line 525, in loads
    return _default_decoder.decode(s)
  File "C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\simplejson\decoder.py", line 370, in decode
    obj, end = self.raw_decode(s)
  File "C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\simplejson\decoder.py", line 400, in raw_decode
    return self.scan_once(s, idx=_w(s, idx).end())
simplejson.errors.JSONDecodeError: Expecting value: line 1 column 1 (

C009821988,682/871
[['1212705751', '1212705775'], ['1212705751', '1212705763'], ['1212705774', '1212705775'], ['1212705774', '1212705763'], ['1212705760', '1212705775'], ['1212705760', '1212705763'], ['1212705768', '1212705775'], ['1212705768', '1212705763'], ['1212705773', '1212705775'], ['1212705773', '1212705763'], ['1212842996', '1212705775'], ['1212842996', '1212705763'], ['1212705762', '1212705775'], ['1212705762', '1212705763'], ['1212705765', '1212705775'], ['1212705765', '1212705763'], ['1212705764', '1212705775'], ['1212705764', '1212705763'], ['1212705770', '1212705775'], ['1212705770', '1212705763'], ['1212705769', '1212705775'], ['1212705769', '1212705763']]
C009844034,683/871
[['1355941284', '1355941287'], ['1355941284', '1355941285'], ['1355941282', '1355941287'], ['1355941282', '1355941285'], ['1355941309', '1355941287'], ['1355941309', '1355941285'], ['1355941283', '1355941287'], ['1355941283', '1355941285'], ['1355941290', '1355941287'], ['1355941290', '1355941285'], 

In [ ]:
getqpl('W009948296')

In [37]:
text = soup.find_all('script', type='text/javascript')
if "application" in dict(json.loads(text[-1].string[29:-1])):
    application = json.loads(text[-1].string[29:-1])["application"]
else:
    application = json.loads(
        text[-1].string[29:-1])["temp-application"]
print(application)

{'id': 'sf-ui-product-details::application', 'key': '', 'loggerName': '@wayfair/sf-ui-product-details', 'pageLevelTags': {'application_name': 'sf-ui-product-details', 'application_key': 'no-application-key', 'brand': 'wayfair', 'store': 'wayfair_com', 'browser': 'chrome', 'client_hostname': 'stores-dsm1-868h.c.wf-gcp-us-storefront-prod.internal', 'datacenter': 'dsm1', 'device': 'desktop', 'locale': 'en-us', 'is_bot': False, 'ssr_status': 'ssr'}, 'useDogStatsD': False, 'useSfpPerfForwarding': False, 'shouldForwardToFS': False, 'shouldSendSessionDataToFS': True, 'ssr': True, 'props': {'enableSSR': True, 'csrfToken': '7423BEFEE1D613F1AA1A802B314BCB95', 'csrfTokenParam': '_csrf_token', 'applicationContext': {'ACTIVE_THEME': 'wayfair', 'ALL_MODERN_ID': 81, 'APPLICATION': 'stores', 'B2B_LEVEL_NAME': 'none', 'BIRCH_LANE_ID': 422, 'CDN_URL': 'https://secure.img1-fg.wfcdn.com/', 'CSRF_PARAM': '_csrf_token', 'CSRF_TOKEN': '7423BEFEE1D613F1AA1A802B314BCB95', 'CURRENT_PATH': '/furniture/pdp/wade-l

In [38]:
prop = application["props"]

flagServiceFlagData = prop["mainCarousel"]["flagServiceFlagData"]

In [41]:
print(flagServiceFlagData)
flag_list = []
if flagServiceFlagData:
    for i in flagServiceFlagData:
        if flagServiceFlagData[i]['flagText'] == 'Sale':
            flag_list.append(i.replace('_','%2C'))
else:
    sale_tag = '-'

{'607493847_634007486': {'flagText': 'Sale', 'flagType': 'sale', 'id': 4, 'rank': 4}, '575875369_607627541': {'flagText': 'Sale', 'flagType': 'sale', 'id': 4, 'rank': 4}, '607493847_616347499': {'flagText': 'Sale', 'flagType': 'sale', 'id': 4, 'rank': 4}, '607493847_634007424': {'flagText': 'Sale', 'flagType': 'sale', 'id': 4, 'rank': 4}, '607493847_616347496': {'flagText': 'Sale', 'flagType': 'sale', 'id': 4, 'rank': 4}, '575875368_607627541': {'flagText': 'Sale', 'flagType': 'sale', 'id': 4, 'rank': 4}, 'W004793865': {'flagText': 'Sale', 'flagType': 'sale', 'id': 4, 'rank': 4}, '607627541_624504988': {'flagText': 'Sale', 'flagType': 'sale', 'id': 4, 'rank': 4}, '607627541_616347497': {'flagText': 'Sale', 'flagType': 'sale', 'id': 4, 'rank': 4}, '607493846_617262834': {'flagText': 'Sale', 'flagType': 'sale', 'id': 4, 'rank': 4}, '582832369_607493846': {'flagText': 'Sale', 'flagType': 'sale', 'id': 4, 'rank': 4}, '607627541_616347494': {'flagText': 'Sale', 'flagType': 'sale', 'id': 4, 

In [42]:
print(flag_list)

['607493847%2C634007486', '575875369%2C607627541', '607493847%2C616347499', '607493847%2C634007424', '607493847%2C616347496', '575875368%2C607627541', 'W004793865', '607627541%2C624504988', '607627541%2C616347497', '607493846%2C617262834', '582832369%2C607493846', '607627541%2C616347494', '607493846%2C617262836', '575875366%2C607627541', '582832115%2C607493846', '581457944%2C607493846', '607493846%2C616347498', '607493847%2C634007425', '607493846%2C617262835']


In [ ]:
with open('package.json', 'r') as f:
    f=f.read()
    f=json.loads(f)
    print(type(f))
    options = f['application']['props']['options']['standardOptions'][0]['options']
    for option in options:
        print("颜色为： {}，piid为：{}".format(option['name'],option['option_id']))


In [9]:
from Wayfair_Scrapy import *
ua = get_ua()
cookie = 'session-id=141-4947585-1612445; session-id-time=2082787201l; i18n-prefs=CAD; ubid-acbca=131-3780108-1716154; lc-acbca=en_CA; session-token="/b5yIxCkGdlmvlk6PohWph570jVYqzG0ia57g8bAdlmw+LZrPJ7WPoxGC4gBtwxi3at58cXF6ya7NTMTYbmkVrKPZxO2w5qS2EqhFjfT7k+6mNzH3UihdjMTOS9rPJQhuP5hHQQVNJ5/zCsw4zQcjOlJ5uKRjNwPbRR+fjw4hOU+gTfapuN2jY94Ii+DDaX09+Nxiu/YSy6NsJB4T4rHNOfBH8OGJMLvVLGPtOTtvfk="; csm-hit=tb:AAXEEJG2Y42WFR6SEB93+s-9GPMW6X58WWX0VVRW6R9|1670220949897&t:1670220949897&adb:adblk_no'
url = 'https://www.amazon.ca/dp/B08TM2W59Y'
headers = {
    'cookie': cookie,
    'user-agent': ua,
    'upgrade-insecure-requests': '1'
}
sp = requests.session().get(url, headers=headers)
content = sp.content
soup = BeautifulSoup(content, "html.parser")
title = soup.find_all(
    'span', class_="a-price")
print(title[0].find_all('span',class_='a-offscreen')[0].text)

$179.99


In [27]:
from Wayfair_Scrapy import *
data = read_src('/Users/huzhang/Nutstore Files/「晓望集群」/S数据分析/Amazon爬虫/SKU_list_US.csv')
ua = get_ua()
cookie_pool = ['session-id=132-3025649-3531568; i18n-prefs=USD; skin=noskin; ubid-main=134-8454797-8017726; session-id-time=2082787201l; lc-main=en_US; session-token="lhWi+dt6/w6Lg1FZDvz31UhbeBndWzDJ970oI1D/oUUzHdke2jOFhplgflWhtbY3/tUAJ0iuOJlBU4f/1T+9OYhZYvOW5ChxDSDwBKv4pZn1dfhipL/eUGO9iZx98QwcnG6AgbVQT+Z1jMaN6wgCcK8fjIWZXauoJ9ZlPsiSzPOfy4Y2D9x58OM7qaVMVwpRSL0cQ6WrXWyIThVRT5rJiN7bJqGQGP4+KRurkVPj10Q="; csm-hit=adb:adblk_no&t:1670234595776&tb:N3QVYE3M3DRFENCJ4GR8+s-TA4PW3KZY9H44TZKEB9Z|1670234595776','session-id=146-2304221-2348360; i18n-prefs=USD; ubid-main=131-6443971-2173464; lc-main=en_US; __utmz=194891197.1669039963.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); session-id-time=2082787201l; skin=noskin; __utmc=194891197; __utma=194891197.829349225.1669039963.1670091843.1670236948.19; session-token="Xs++Fl6jryRDGa80ZJM1wjoiceptmn5swmUHTmOU7g/UIQU1THHwuszMryWIb/cJVZWtdL3MjN4fkiyN8WAWd3Ln/D3etPa+/RYEKU/+HQXrBPHbkHya87h0mTHHw4LABfrss5A3BNj6bJE37mONOJwBrPmwGqaq8RqXNlGO1a7/mk4etcyCG0bJK4qR0ylw/DAl5jRXR4wtrX8aEZhuXcq3lfsoYByS2tYelxADHjc="; csm-hit=tb:s-NARNVWXJPW8WBW30XJFZ|1670311889055&t:1670311892111&adb:adblk_no','session-id=135-9202731-5638555; i18n-prefs=USD; ubid-main=133-0791837-8912640; lc-main=en_US; session-id-time=2082787201l; session-token="gxB5fBZdvqJBqcl6ydEbU5KpBViljwq2w+xy9WAFHlLLS+dOpbworx6KKO0EsTRiaEZc8SpVSvjBVXNxhE9vodG185QhBORZK1tdT+dTkpqhISF0pBJd9/v4E2N8jvQdjtogmXaYWntrWxXahsrPvExBfxl2wv/rFoq3DTOmvvNn3al1NlyUmAF45yNz4eckJRMN88WkeFQRMVL+17Km+aeofPodjPmdLDjCF/LV5hg="; sp-cdn="L5Z9:HK"; csm-hit=tb:s-72W4QNP5BKJDRVGXT50W|1670311961435&t:1670311963286&adb:adblk_no']
cookie = 'session-id=132-3025649-3531568; i18n-prefs=USD; skin=noskin; ubid-main=134-8454797-8017726; session-id-time=2082787201l; lc-main=en_US; session-token="g7reWoH9uzHvllMfIcP2v3qUY2nR2izGuac/GUnKvTleF3ArXEOFyRmnWVAxRheED56LBEQiacReWM5U4ng6WXOxQc1FKaUnm+fcU1n7XO0DLXdk0PbP7zaLrHSp6UeBKw1LtOy2TBBp9/zoy/U2MgNcN7k5QexvEHSzYCNKY45wSrDEnxAGLmG26XmSFYsi1ezsJLW4t0Uu125u8VQjNMa85RsnahVsM3HxtUYLBHY="; csm-hit=adb:adblk_no&t:1670316849288&tb:N3QVYE3M3DRFENCJ4GR8+s-1EHXY0SC7EJSKTQCXY31|1670316849288'

proxy = '221.131.141.243:9091'
dict1 = {'Asin':[],'Sale_price':[],'title':[],'rating':[],'star':[]}
for i in tqdm(range(len(data))):
    headers = {
        'cookie': cookie,
        'user-agent':get_ua(),
        'upgrade-insecure-requests': '1'
    }
    asin = data[i][0]
    url = 'https://www.amazon.com/gp/product/ajax/ref=auto_load_aod?asin={}&experienceId=aodAjaxMain'.format(asin)
    sp = requests.session().get(url, headers=headers)
    content = sp.content
    soup = BeautifulSoup(content, "html.parser")
    limit = 0
    while soup.title is not None:
        if limit>20:
            break
        else:
            limit+=1
        headers = {
            'cookie': cookie,
            'user-agent':get_ua(),
            'upgrade-insecure-requests': '1'
        }
        asin = data[i][0]
        url = 'https://www.amazon.com/gp/product/ajax/ref=auto_load_aod?asin={}&experienceId=aodAjaxMain'.format(asin)
        sp = requests.session().get(url, headers=headers)
        content = sp.content
        soup = BeautifulSoup(content, "html.parser")
    if soup.find_all('span',class_='a-offscreen'):
        sale_price = soup.find_all('span',class_='a-offscreen')[0].text.strip()
    else:
        sale_price = '-'
        print(sale_price)
    if soup.find_all('span',class_='a-offscreen'):
        sale_price = soup.find_all('span',class_='a-offscreen')[0].text.strip()
    else:
        sale_price = '-'
        print(sale_price)
    if soup.find_all('h5',class_='aod-asin-title-text-class'):
        title = soup.find_all('h5',class_='aod-asin-title-text-class')[0].text.strip()
    else:
        title = '-'
    if soup.find_all('span',id='aod-asin-reviews-count-title'):
        rating = soup.find_all('span',id='aod-asin-reviews-count-title')[0].text.strip()
    else:
        rating = '-'
    if soup.find_all('i',id='aod-asin-reviews-star'):
        star = soup.find_all('i',id='aod-asin-reviews-star')[0]['class'][-1][7:]
    else:
        star = '-'
    dict1['Asin'].append(asin)
    dict1['Sale_price'].append(sale_price)
    dict1['title'].append(title)
    dict1['rating'].append(rating)
    dict1['star'].append(star)


 15%|█▍        | 108/744 [02:57<16:17,  1.54s/it]

失败
失败


 68%|██████▊   | 504/744 [14:19<31:29,  7.87s/it]

失败
失败


 77%|███████▋  | 574/744 [20:23<05:57,  2.10s/it]

...

 80%|████████  | 597/744 [21:56<13:40,  5.58s/it]

失败
失败


 98%|█████████▊| 727/744 [32:49<01:46,  6.26s/it]

失败
失败


100%|██████████| 744/744 [33:52<00:00,  2.73s/it]


In [28]:
print(dict1)

{'Asin': ['B09L82QXCC', 'B09L83ND1Y', 'B09L89C21G', 'B09L87DN9Z', 'B0BHY8NPSK', 'B0BBR2P8D6', 'B0BBMXLZ9X', 'B0BBMKS8G8', 'B0BBMNXWLC', 'B0BBR47VN4', 'B0BBR3RZ2L', 'B0BBR4CMKG', 'B08PBJBBRQ', 'B09F9PT6M3', 'B09FLG1T8Q', 'B07MPGG4S3', 'B097DMZGW3', 'B0BHNVQL9J', 'B0BHNZ3671', 'B097DKP7WV', 'B0BCKGY1N7', 'B0BCKJ32L6', 'B0BG24XCHN', 'B08PBZX7Z1', 'B0BKQ3KZTJ', 'B0BKQCZ5T7', 'B09S5F1VHQ', 'B08PBZ48T9', 'B09M3VHRZB', 'B09M3WZMRX', 'B09MQSPP2C', 'B0BHYTDP7X', 'B08PBXWDB3', 'B0BD7TN5L1', 'B0BD7W4R7R', 'B0BD7W51D4', 'B09H2LKC9X', 'B09LYY3X7D', 'B0BBGN8KKF', 'B0BBGSZYF4', 'B0BBGSP7F6', 'B08JTTS18K', 'B08JSHGD8X', 'B08JTP3QM8', 'B0BLSSDJ8C', 'B0BLSRLF6Z', 'B09KGX4XZX', 'B08LPFQC64', 'B08LQ7C9VG', 'B07QJ4VJ3Q', 'B07JJK3JVP', 'B0B77HCSLV', 'B09PV5YCZM', 'B0BB7LYTH3', 'B0BB7KRFCM', 'B0BB7HG2WD', 'B0BB7HF6LC', 'B0BB7N1RKF', 'B0BB7KLTDJ', 'B0BB7M87JR', 'B0BB7M6G4G', 'B0BB7HM5QG', 'B0BB7NXHQ5', 'B0BB7ML6CZ', 'B0BB7M5JYW', 'B0BB7K34QK', 'B0BHVMF2WT', 'B08G4CZ68F', 'B09HBX93MX', 'B07N68CND1', 'B08LD4SL1

In [32]:
df = pd.DataFrame(dict1)
df.to_csv('output.csv',encoding='gb18030',index_label=None)

In [ ]:
video link position
application.props.mainCarousel.videos[i].sources[0].source

In [ ]:
application.props.delivery.is_out_of_stock  false

In [19]:
import requests
j = requests.get('https://secure.img1-fg.wfcdn.com/webpack/sf-ui-product-details/bundles/wayfair/en-us/js/application.32dc6ddb.bundle.js')
j = j.content.decode()
print(type(j))
print(j.find('a636f23a2ad15b342db756fb5e0ea093'))
print(j[1562643:1568643])

<class 'str'>
1564659
query reviewQuery($sku: String!, $sort_order: Reviews_sort_order_enum, $page_number: Int, $filter_rating: String, $reviews_per_page: Int, $has_photos: Boolean, $search_query: String, $language_code: String!) {\n product(sku: $sku) {\n customerReviews: customer_reviews(\n sort_order: $sort_order\n page: $page_number\n filter_rating: $filter_rating\n reviews_per_page: $reviews_per_page\n has_photos: $has_photos\n search_query: $search_query\n ) {\n ratingCount: rating_count\n averageRatingValue: average_rating_value\n histogramStats: histogram_stats {\n rating\n count\n __typename\n }\n filterAspects: aspects {\n aspect_name\n aspect_count\n __typename\n }\n bubbleFilters: bubble_filters {\n id\n adjective\n aspect\n sku\n occurrences\n reviewPreviewId\n __typename\n }\n reviewPagesTotal: review_pages_total\n reviews {\n reviewId: review_id\n reviewerName: reviewer_name\n hasVerifiedBuyerStatus: has_verified_buyer_status\n isUSReviewer: is_us_reviewer\n reviewerBadg

In [ ]:
load = "https://www.wayfair.com/a/inventory/load"

In [ ]:
video link position
application.props.mainCarousel.videos[i].sources[0].source

In [ ]:
application.props.delivery.is_out_of_stock  false